📘 Example: Creating a Historical Event in FloodAdapt

This notebook demonstrates how to create a synthetic event using FloodAdapt. Historical events are valuable for controlled testing, sensitivity analysis, and understanding the behavior of flood models under simplified or hypothetical scenarios. 

A FloodAdapt Event consists of 2 things:
- a TimeFrame describing the start and end time of the hazard simulation(s)
- a collection of forcings to be applied to the hazard model(s)

In this example, we construct a full historical event with water level, rainfall, wind, and river discharge forcings, and then save it to a FloodAdapt database.

⏱️ 1. Setup and Imports

We begin by importing the required classes and modules for constructing synthetic forcings and managing event data within the flood_adapt framework.

In [ ]:
import flood_adapt.objects.forcing as f

from pathlib import Path
from datetime import datetime

from flood_adapt.objects import HistoricalEvent, TimeFrame
from flood_adapt import unit_system as us
from flood_adapt import FloodAdapt, Settings
from flood_adapt.config.sfincs import RiverModel

# Configure database
Settings(
    DATABASE_ROOT=Path("../../_data/examples").resolve(),
    DATABASE_NAME="charleston_test"
)

🗓️ 2. Define the Simulation Time Frame

We specify a one-day time frame for the synthetic event, from January 1 to January 2, 2025.

In [ ]:
# Create an time frame for the simulation
start_time = datetime(year=2020, month=1, day=1)
end_time = datetime(year=2020, month=1, day=2)
time_frame = TimeFrame(start_time=start_time, end_time=end_time)

🌊 3. Define Water Level Forcing

Synthetic water levels are constructed from a combination of tidal and surge components.

In [ ]:
# Recorded water levels from a CSV file
csv_file = Path("../../_data/tide.csv").resolve()
water_levels = f.WaterlevelCSV(path=csv_file)
wl_df = water_levels.to_dataframe(time_frame=time_frame)

# Alternative: Water levels downloaded from a tide gauge
tide_gauge = f.TideGauge(
    name=8665530,
    description="Charleston Cooper River Entrance",
    source="noaa_coops",
    reference="MSL",
    ID=8665530,
    lat=32.78,
    lon=-79.9233,
    units="meters",
)
df_tide_gauge = tide_gauge.get_waterlevels_in_time_frame(
    time=time_frame,
)
water_levels_gauged = f.WaterlevelGauged()

# Alternative: Water levels simulated by an offshore model
water_levels_from_offshore = f.WaterlevelModel()

# Inspect
wl_df.plot(
    title="Water Level from CSV",
    xlabel="Time",
    ylabel="Water Level (m)",
    legend=False,
    figsize=(5, 2)
)

df_tide_gauge.plot(
    title="Water Level from Tide Gauge",
    xlabel="Time",
    ylabel="Water Level (m)",
    legend=False,
    figsize=(5, 2)
)

🌧️ 4. Define Meteo Forcing

FloodAdapt provides an easy connection to gfs data, by only requiring the TimeFrame for which to download hindcast data.

In [ ]:
meteo_dataset = f.MeteoHandler().read(time_frame)
print(meteo_dataset) # TODO make sure the meteo files are already downloaded in the database to circumvent flaky noaa coops API calls

### Visualizing Meteo data

It can be difficult to visualize spatially varying timeseries data. So, below is a simple animation generator to do some basic data validation on the downloaded gfs data.

Choose any of the available timeseries data variables and generate the animation.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

to_plot = 'press_msl' # available variables: 'wind10_u' or 'wind10_v' or 'press_msl' or 'precip'

var = meteo_dataset[to_plot] 
fig, ax = plt.subplots()
plot = var.isel(time=0).plot(ax=ax, cmap='viridis', add_colorbar=True)

def update(frame):
    ax.clear()
    var.isel(time=frame).plot(ax=ax, cmap='viridis', add_colorbar=False)
    ax.set_title(f'Time: {str(var.time[frame].values)}')

ani = animation.FuncAnimation(fig, update, frames=len(var.time), interval=200)
plt.close(fig)

HTML(ani.to_jshtml())

🌬️ 5. Define Meteo Forcings

To use the downloaded gfs hindcast data in FloodAdapt, you need to create Meteo forcings and add them to your event.

Under the hood, the meteo forcings use the MeteoHandler to download the data, and then return slices of that dataset.

In [ ]:
rainfall = f.RainfallMeteo()
wind = f.WindMeteo()

🏞️ 6. Define River Discharge Forcing

Discharge is defined for two pre-configured rivers. These rivers must be registered in the hazard model configuration beforehand.

In [ ]:
# Add discharge
# The available rivers are defined in the hazard model when creating the database.
# You cannot add new rivers to the model in an event, you can only set the discharge of each given river.
river1 = RiverModel(
    name="River1",
    x_coordinate=1,
    y_coordinate=1,
    mean_discharge=us.UnitfulDischarge(value=100, units=us.UnitTypesDischarge.cms), 
    # Mean discharge is used create default discharge values for the river, you can still overwrite it later.
)
river2 = RiverModel(
    name="River2",
    x_coordinate=2,
    y_coordinate=2,
    mean_discharge=us.UnitfulDischarge(value=200, units=us.UnitTypesDischarge.cms),
    # Mean discharge is used create default discharge values for the river, you can still overwrite it later.
)

discharge1 = f.DischargeConstant(
    river=river1,
    discharge=us.UnitfulDischarge(value=100, units=us.UnitTypesDischarge.cms)
)
discharge2 = f.DischargeConstant(
    river=river2,
    discharge=us.UnitfulDischarge(value=200, units=us.UnitTypesDischarge.cms)
)

# Inspect
df1 = discharge1.to_dataframe(time_frame=time_frame)
df2 = discharge2.to_dataframe(time_frame=time_frame)
df_combined = df1.join(df2)
df_combined.plot(title="Constant Discharge River", xlabel="Time", ylabel="Discharge (cms)", legend=True, figsize=(5, 2))

🧩 7. Combine Forcings and Create Synthetic Event

All defined forcings are collected into a single dictionary, which is used to construct a SyntheticEvent.

In [ ]:
# Create the forcings dictionary
# The keys of the dictionary are the forcing types, and the values are lists of the corresponding forcing objects.
forcings = {
    f.ForcingType.WATERLEVEL: [water_levels], # or one of `water_levels_gauged` or `water_levels_from_offshore`,
    f.ForcingType.RAINFALL: [rainfall],
    f.ForcingType.WIND: [wind],
    f.ForcingType.DISCHARGE: [discharge1, discharge2],
}

# Create a SyntheticEvent with the forcings and time frame
event = HistoricalEvent(
    name="example_historical_event",
    time=time_frame,
    forcings=forcings,
)

💾 8. Save the Event to a FloodAdapt Database

Finally, we save the event to a FloodAdapt database.

In [ ]:
# Save the event to the database
# fa = FloodAdapt(database_path=Path("path/to/database")) # TODO
# fa.save_event(event=event)